In [ ]:
!pip install gsd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 623.6/623.6 kB 6.7 MB/s eta 0:00:00


In [ ]:
!pip install numba

In [ ]:
import numpy as np
import gsd.hoomd
import os
import numba as nb
import matplotlib.pyplot as plt
import numpy as np
import gsd.hoomd
import multiprocessing
import pandas as pd
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp
import joblib
from tqdm import tqdm
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
from google.colab import drive

try:
  drive.mount('/content/drive', force_remount=True)
except PermissionError:
  print("Permission error occurred, rerun the code to grant permission.")
  raise

Mounted at /content/drive


In [ ]:
# utils

def calc_angle_mid(v1, v2):
    v1 /= np.linalg.norm(v1)
    v2 /= np.linalg.norm(v2)
    cos_angle = np.dot(v1, v2)
    cos_angle = np.clip(cos_angle, -1.0, 1.0)
    return np.arccos(cos_angle)

def calc_cos_mid(v1, v2):
    v1 /= np.linalg.norm(v1)
    v2 /= np.linalg.norm(v2)
    cos_angle = np.dot(v1, v2)
    return cos_angle

def distances(coordinates, box_size):
    natoms = coordinates.shape[0]
    coordinates = np.tile(coordinates, (natoms, 1, 1))
    diff = coordinates - coordinates.transpose((1, 0, 2))
    tij = box_size * np.round(diff / box_size)
    rij = diff - tij
    dist_matrix = np.sqrt(np.sum(rij**2, axis=2))
    return dist_matrix

# potentials

# use 100 local slatm instead of 320

def get_boa(z1, nuclear_charges):
    count = (nuclear_charges == z1).sum()
    return z1 * np.array([count])

def get_sbop(z1, coordinates, box, normalize=True, rcut=4.0, dgrid=0.03, sigma=0.01, rpower=6):

    r0, r1 = 0.1, rcut
    nx = int((rcut - r0)/dgrid) + 1
    xs = np.linspace(r0, r1, nx)
    coeff = 1/np.sqrt(2*sigma**2*np.pi) if normalize else 1.0
    c0 = z1**2 * coeff
    xs0 = c0/(xs**rpower) * dgrid
    inv_sigma = -1.0 / (sigma**2)

    natoms = coordinates.shape[0]
    ys = np.zeros(nx)

    diff = coordinates[:,None,...] - coordinates[None,...]
    rij = diff - box * np.round(diff/box)
    dist2 = np.sqrt(np.sum(rij**2, axis=-1))
    k = np.round((dist2 - r0) / dgrid).astype(int)
    k[(k >= nx) | (k < 0)] = 0
    w = np.exp(inv_sigma * (xs[k] - dist2)**2)
    mask = (dist2 != 0) & (dist2 <= rcut) & (k < nx) & (k > 0)
    xs0_w = xs0[k] * w * mask
    ys = np.bincount(k.flatten(), weights=xs0_w.flatten())
    return ys * 2

# threebody jit

@nb.njit
def calc_angle_mid(v1, v2):
    norm_v1 = np.sqrt(np.sum(v1**2))
    norm_v2 = np.sqrt(np.sum(v2**2))
    v1 /= norm_v1
    v2 /= norm_v2
    cos_angle = np.dot(v1, v2)

    cos_angle = -1.0 if cos_angle < -1.0 else (1.0 if cos_angle > 1.0 else cos_angle)

    return np.arccos(cos_angle)


@nb.njit
def calc_cos_mid(v1, v2):
    norm_v1 = np.sqrt(np.sum(v1**2))
    norm_v2 = np.sqrt(np.sum(v2**2))
    v1 /= norm_v1
    v2 /= norm_v2
    cos_angle = np.dot(v1, v2)
    return cos_angle

@nb.njit(parallel=True)
def distances(coordinates, box_size):
    natoms = coordinates.shape[0]
    dist_matrix = np.empty((natoms, natoms), dtype=np.float64)

    for i in nb.prange(natoms):
        for j in range(natoms):
            diff = coordinates[i] - coordinates[j]
            tij = box_size * np.round(diff / box_size)
            rij = diff - tij
            dist_matrix[i, j] = np.sqrt(np.sum(rij**2))

    return dist_matrix


@nb.njit
def get_sbot(z, coordinates, box, normalize=True, rcut=4.8, dgrid=0.03, sigma=0.05):
    dist_matrix = distances(coordinates, box)
    d2r = np.pi / 180.0
    a0, a1 = -20.0 * d2r, np.pi + 20.0 * d2r
    nx = int((a1 - a0)/dgrid) + 1
    xs = np.empty(nx)
    for i in range(nx):
        xs[i] = a0 + i * dgrid

    prefactor = 1.0 / 3.0
    coeff = 1/np.sqrt(2*sigma**2*np.pi) if normalize else 1.0
    c0 = prefactor * (z % 1000)**3 * coeff * dgrid
    inv_sigma = -1.0 / (2*sigma**2)

    cos_xs = np.cos(xs) * c0

    natoms = coordinates.shape[0]
    ys = np.zeros(nx)

    ias = np.arange(natoms)
    mask = (dist_matrix < rcut) & (dist_matrix > 0)

    for i in range(80):
        for j in range(natoms):
            if i == j:
                continue

            rij = dist_matrix[i, j]

            if rij > rcut:
                continue

            for k in range(natoms):
                if k == i or k == j:
                    continue

                if not mask[i, k] or not mask[j, k]:
                    continue

                diff3 = coordinates[j] - coordinates[k]
                tik = box * np.round(diff3 / box)
                rjk = diff3 - tik
                r3 = np.sqrt(np.sum(rjk**2))

                if r3 > rcut:
                    continue

                diff1 = coordinates[i] - coordinates[j]
                tij = box * np.round(diff1/box)
                rij = diff1 - tij

                diff2 = coordinates[i] - coordinates[k]
                tik = box * np.round(diff2/box)
                rik = diff2 - tik

                ang = calc_angle_mid(rij, -rjk)
                cok = calc_cos_mid(rik, rjk)
                coi = calc_cos_mid(-rij, -rik)

                r1 = dist_matrix[i, j]
                r2 = dist_matrix[i, k]
                r3 = dist_matrix[j, k]

                r = r1 * r2 * r3

                ys += (c0 + cos_xs*cok*coi)/(r**3) * np.exp((xs-ang)**2 * inv_sigma)

    return ys / 2

In [ ]:
# process trajectories

def vectors_to_file(vectors, base_filename, output_suffix):
  output_path = f"/content/drive/MyDrive/current_05_08/pbcs-extended/{base_filename}{output_suffix}.txt"
  print(f"Saving file to: {output_path}")
  with open(output_path, 'w') as f:
    for vector in vectors:
      np.savetxt(f, [vector], fmt='%.6f')

def ensemble_avg_weighted(traj):
    sum_array = np.sum(traj, axis=0)
    vector_traj = sum_array / len(traj)
    weights = np.array([0.004] + [0.504]*131 + [0.492]*128)  #dgrid 0.03 case. needs generalization.
    avg_ens = weights * vector_traj
    return avg_ens

def process_frame(frame):
    try:
        coord = frame.particles.position
        box = frame.configuration.box[:3][1]
        n_atoms = len(coord)
        charges = np.ones(n_atoms)
        onebody = get_boa(1, charges)
        twobody = get_sbop(1, coord, box, normalize=True, rcut=4.0, dgrid=0.03, sigma=0.05, rpower=6)
        threebody = get_sbot(1, coord, box, normalize=True, rcut=4.0, dgrid=0.03, sigma=0.05)
        mbs = [onebody, twobody, threebody]
        vector = np.concatenate(mbs, axis=0)
        return vector
    except Exception as e:
        logging.error(f"Error processing frame: {e}")
    return None

def vectors_per_sim(trajectory):
    try:
        traj = gsd.hoomd.open(trajectory, mode='r')
        num_frames = len(traj)
        start_frame = max(0, num_frames - 20)

        with ProcessPoolExecutor() as executor:
            frames = list(traj[start_frame:])
            vectors = list(tqdm(executor.map(process_frame, frames), total=len(frames)))
        return vectors
    except Exception as e:
        logging.error(f"Error processing trajectory {trajectory}: {e}")
        return None

In [ ]:
# run

def main():
    num_files = 60
    start_file = 271
    trajectory_filenames = [f"/content/drive/MyDrive/current_05_08/extended_traj/traj{i}.gsd" for i in range(start_file, num_files + start_file)]

    for trajectory in trajectory_filenames:
        print(f"Processing trajectory: {trajectory}")
        vectors = vectors_per_sim(trajectory)
        if vectors:
          avg_ens = ensemble_avg_weighted(vectors)
          filename = os.path.splitext(os.path.basename(trajectory))[0]
          vectors_to_file(avg_ens, filename, '-slatm-pbcs-ext')

if __name__ == "__main__":
    main()

Processing trajectory: /content/drive/MyDrive/current_05_08/extended_traj/traj271.gsd


100%|██████████| 20/20 [04:47<00:00, 14.39s/it]


Saving file to: /content/drive/MyDrive/current_05_08/pbcs-extended/traj271-slatm-pbcs-ext.txt
Processing trajectory: /content/drive/MyDrive/current_05_08/extended_traj/traj272.gsd


100%|██████████| 20/20 [01:46<00:00,  5.32s/it]


Saving file to: /content/drive/MyDrive/current_05_08/pbcs-extended/traj272-slatm-pbcs-ext.txt
Processing trajectory: /content/drive/MyDrive/current_05_08/extended_traj/traj273.gsd


...

Saving file to: /content/drive/MyDrive/current_05_08/pbcs-extended/traj328-slatm-pbcs-ext.txt
Processing trajectory: /content/drive/MyDrive/current_05_08/extended_traj/traj329.gsd


100%|██████████| 20/20 [05:55<00:00, 17.75s/it]


Saving file to: /content/drive/MyDrive/current_05_08/pbcs-extended/traj329-slatm-pbcs-ext.txt
Processing trajectory: /content/drive/MyDrive/current_05_08/extended_traj/traj330.gsd


100%|██████████| 20/20 [03:33<00:00, 10.68s/it]

Saving file to: /content/drive/MyDrive/current_05_08/pbcs-extended/traj330-slatm-pbcs-ext.txt
